У додатку ви знайдете файл nyt.png, що містить візуалізацію від видання The New York Times. Ваше завдання - відтворити цю візуалізацію максимально близько до оригіналу.
https://www.nytimes.com/interactive/2018/01/18/climate/hottest-year-2017.html
Ви можете використовувати будь-який доступний вам інструмент.

Якщо ви використовуєте код, я очікую від вас посилання на GitHub репозиторій із картинкою та кодом. Якщо ви використовуєте щось на зразок Tableau або Power BI, я очікую від вас посилання на опубліковану візуалізацію. 

Зверніть увагу, що у візуалізації The New York Times дані наводяться у градусах Цельсія, але в даних у csv файлі - ні. Не варто на цьому етапі переводити одну розмірність в іншу, просто відтворіть структуру візуалізації на основі цих даних. Також зверніть увагу, що вам потрібно буде трохи трансформувати дані (в Pandas це можна зробити за допомогою методів wide to long або melt)

![alt](predicted.png "Image")

In [1]:
import sys
!{sys.executable} -m pip install pandas altair

In [223]:
import pandas as pd
import altair as alt

# setting needed theme
alt.themes.enable("fivethirtyeight")

ThemeRegistry.enable('fivethirtyeight')

In [224]:
# importing data
df = pd.read_csv('data.csv', skiprows=[0])
df.head()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,J-D,D-N,DJF,MAM,JJA,SON
0,1880,-0.29,-0.18,-0.11,-0.19,-0.11,-0.23,-0.21,-0.09,-0.16,-0.23,-0.20,-0.23,-0.19,***,***,-0.14,-0.18,-0.20
1,1881,-0.15,-0.17,0.04,0.04,0.02,-0.20,-0.06,-0.02,-0.13,-0.20,-0.21,-0.10,-0.10,-.11,-.18,0.03,-0.09,-0.18
2,1882,0.15,0.15,0.04,-0.18,-0.16,-0.26,-0.20,-0.05,-0.10,-0.24,-0.16,-0.24,-0.10,-.09,.06,-0.10,-0.17,-0.17
3,1883,-0.31,-0.39,-0.13,-0.17,-0.20,-0.12,-0.08,-0.15,-0.20,-0.14,-0.22,-0.16,-0.19,-.20,-.31,-0.16,-0.12,-0.19
4,1884,-0.15,-0.08,-0.37,-0.42,-0.36,-0.40,-0.34,-0.26,-0.27,-0.24,-0.30,-0.28,-0.29,-.28,-.13,-0.39,-0.34,-0.27


In [225]:
# calculating average temperature by year in separate column
df['average_temp'] = df.iloc[:, [1,2,3,4,5,6,7,8,9,10,11,12]].mean(axis=1)
# dropping all columns, except for year and average temp
df.drop(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'J-D', 'D-N', 'DJF', 'MAM', 'JJA', 'SON'], axis=1, inplace = True)
df.head()                             

,Year,average_temp
0,1880,-0.185833
1,1881,-0.095000
2,1882,-0.104167
3,1883,-0.189167
4,1884,-0.289167


In [226]:
# calculating average t for 19 century
average_temp_19_cent = df.loc[df['Year'] < 1900].mean().average_temp
print(average_temp_19_cent)

-0.22804166666666664


In [227]:
# making this average as 0 of our graph
df['average_temp'] += -average_temp_19_cent

In [228]:
# magic
df["Year_str"] = df["Year"]
df = df.astype({"Year_str": str})

In [229]:
def gen_text(x, y, text, font_size, font_weight="normal"):
    """
    Function that draws text on chart.
    Made function because it is used often
    """
    return alt.Chart({'values':[{'x': x, 'y': y}]}).mark_text(
            text=text, fontSize=font_size, fontWeight=font_weight
        ).encode(
            x='x:T', y='y:Q'
        )

def gen_condition():
    """Function that makes loong js condition"""
    years_to_show = [1944, 1904, 1998, 2014, 2015, 2016, 2017]
    return " || ".join([f"datum.Year == {elm}" for elm in years_to_show])

# main chart
main_chart = alt.Chart(df).mark_point(color = 'grey', 
                         fill = '#f26c08', 
                         size = 100, 
                         opacity = 1,
                         strokeWidth = 1).encode(
    x = alt.X(field='Year_str',
              type='temporal',
              title = '',
              axis = alt.Axis(tickCount = 14, labelColor="Grey", labelFontSize=13),
              scale=alt.Scale(zero=False)),
    y = alt.Y(field='average_temp',
              type='quantitative',
              title = '',
              axis = alt.Axis(tickCount = 9, labelColor="Grey", labelFontSize=13),
              scale=alt.Scale(zero=False)),
)

# chart that shows year near selected points
points_year_text = main_chart.mark_text(
    align='left',
    baseline='middle',
    dx=7
).encode(
    text=alt.condition(
          gen_condition(),
           "Year:N",
          alt.value("")
      )
)

# all text
top_text_t = gen_text("1901", "1.2", "Annual Global Surface Temperature", 15, "bold")
top_text_b = gen_text("1901", "1.1", "Relative to late 19th Century Average", 15, "bold")
bottom_text_t = gen_text("2008", "0.12", "HOTTER  THAN THE", 12)
bottom_text_m = gen_text("2008", "0.07", "1800-1899 AVERAGE", 12)
bottom_text_b = gen_text("2013", "-0.07", "COLDER", 12)

# horisontal line
rule = alt.Chart(pd.DataFrame({'y': [0]})).mark_rule().encode(y='y')

# show
(main_chart + points_year_text + top_text_t + top_text_b + bottom_text_t + bottom_text_m + bottom_text_b + rule).properties(width = 900, height = 500, background = '#F9F9F9', padding = 25)

alt.LayerChart(...)

Actual result:
![alt](actual.png "Image")